In [5]:
'''
Created by Oscar Scholin and Graham Hirsch
--> Reads in HCV dataframe, extracts lightcurve data for each object, in each filter combo
--> outputs a vector per matchid with the 25 variability metrics identified by https://arxiv.org/pdf/1710.07290.pdf
'''

'\nCreated by Oscar Scholin and Graham Hirsch\n--> Reads in HCV dataframe, extracts lightcurve data for each object, in each filter combo\n--> outputs a vector per matchid with the 25 variability metrics identified by https://arxiv.org/pdf/1710.07290.pdf\n'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import statistics
from statistics import median, mean, stdev
import time

### Load HCV

In [2]:
#set current working directory (cwd)
os.chdir('/Users/oscarscholin/Desktop/Pomona/Summer22/Quetin/')

In [7]:
#call the dataframe from the hcv csv folder within the cwd
df = pd.read_csv(r'./raw data/hcv/HCV_7_1.csv')
df

,matchid,groupid,subgroup,ra,dec,pipeline_class,expert_class,filter,num_filters,var_quality_flag,...,lightcurve_d,lightcurve_m,lightcurve_cm,lightcurve_e,lightcurve_i,lightcurve_r,ci_d,ci_v,d_d,d_v
0,352,1040910,26,269.790894,-29.248819,1,0,ACS_F606W,2,AAAAC,...,55855.000881,25.631300,25.636757,0.1070,hst_12586_03_acs_wfc_f606w,False,55855.000881,1.093611,55855.000881,6.084347
1,352,1040910,26,269.790894,-29.248819,1,0,ACS_F606W,2,AAAAC,...,56001.959890,26.200701,26.195594,0.1815,hst_12586_10_acs_wfc_f606w,False,56001.959890,0.635000,56001.959890,2.961927
2,352,1040910,26,269.790894,-29.248819,1,0,ACS_F606W,2,AAAAC,...,56052.841962,26.034100,26.046429,0.1469,hst_12586_22_acs_wfc_f606w,False,56052.841962,0.824907,56052.841962,5.407275
3,352,1040910,26,269.790894,-29.248819,1,0,ACS_F606W,2,AAAAC,...,56109.701472,25.735399,25.760180,0.1854,hst_12586_35_acs_wfc_f606w,False,56109.701472,0.933519,56109.701472,4.572665
4,352,1040910,26,269.790894,-29.248819,1,0,ACS_F606W,2,AAAAC,...,56121.406559,25.626101,25.640382,0.0986,hst_12586_39_acs_wfc_f606w,False,56121.406559,1.029815,56121.406559,3.149204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576369,108169792,1063416,12,269.609375,-29.169004,2,0,WFC3_F814W,2,AAABC,...,56716.158792,20.118099,20.122682,0.0061,hst_13463_08_wfc3_uvis_f814w,False,56716.158792,1.415333,56716.158792,2.586158
2576370,108169792,1063416,12,269.609375,-29.169004,2,0,WFC3_F814W,2,AAABC,...,56732.954985,20.060400,20.060412,0.0058,hst_13463_12_wfc3_uvis_f814w,False,56732.954985,1.374222,56732.954985,6.504886
2576371,108169792,1063416,12,269.609375,-29.169004,2,0,WFC3_F814W,2,AAABC,...,56764.960344,20.134701,20.137863,0.0063,hst_13463_20_wfc3_uvis_f814w,False,56764.960344,1.420444,56764.960344,4.001287
2576372,108169792,1063416,12,269.609375,-29.169004,2,0,WFC3_F814W,2,AAABC,...,56784.897984,20.104401,20.095573,0.0057,hst_13463_24_wfc3_uvis_f814w,False,56784.897984,1.412222,56784.897984,3.657708


### Function to output list of lightcurve data for given matchid

In [3]:
#returns list of lists of [matchid, filter, [(time, mag), (time, mag), ..], [MagErr per observation]]
#we will treat each matchid and filter combo as a separate object
def get_lightcurve_data(matchid):
    #get matchid dataframe
    matchid_df = df.loc[df['matchid']==matchid]
    
    #find the matchid in the big df as well for MagErr
    matchid_big_df = big_df.loc[big_df['MatchID']==matchid]
    
    #extract filter types
    matchid_filters = list(matchid_df['filter'].unique())
    #initialize list of [matchid, filter, time,mag tuples]
    lightcurve_data_all = []
    
    #iterate through filter types to return list of lists
    for filter_type in matchid_filters:
        #get a filter, matchid dataframe to extract lightcurve_d, lightcurve_m tuples
        filter_matchid_df = matchid_df.loc[matchid_df['filter']==filter_type]
        time_list = list(filter_matchid_df['lightcurve_d'].values)
        mag_list = list(filter_matchid_df['lightcurve_m'].values)
        
        #zip time and mag lists together
        time_mag_tuples = list(zip(time_list, mag_list))
        
        mag_err_list = list(filter_matchid_df['lightcurve_e'].tolist())
        
        #print('mag list', len(mag_list))
        #print('mag err', len(mag_err_list))
        
        #append info to main list
        lightcurve_data_all.append([matchid, filter_type, time_mag_tuples, mag_err_list])
        
    #return lightcurve_info
    return lightcurve_data_all    

### Calculate variability indices

In [42]:
#MAD--HCV
def get_MAD_HCV(lightcurve_data):
    #extract matchid and filter_type from lightcurve_data
    matchid = lightcurve_data[0]
    filter_type = lightcurve_data[1]
    
    #get df for matchid and filter type
    filter_matchid_df = df.loc[(df['matchid']==matchid) & (df['filter']==filter_type)]
    
    #the HCV already calculated MAD, so pull the value
    mad = filter_matchid_df['mad'].values[0]
    return mad

#MAD--custom
def get_mad(lightcurve_data):
    #extract matchid and filter_type from lightcurve_data
    matchid = lightcurve_data[0]
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
    
    #compute median of mag_list
    median_mag = median(mag_list)
    
    #now compute absolute deviations from the median
    absolute_deviations_list = []
    for mag in mag_list:
        absolute_deviations_list.append(abs(mag - median_mag)) 
        
    #compute median of the absolute deviations; this is MAD!
    mad = median(absolute_deviations_list)
    
    return mad

#helper function to calculate weighted mean
def get_weighted_mean(mag_list, mag_err_list):
    #create empty lists to sum at end
    weighted_mean_num_list = []
    weighted_mean_denom_list = []
    for i, magerr in enumerate(mag_err_list):
        weighted_mean_num_list.append(
            mag_list[i] / (magerr**2)
        )
        
        weighted_mean_denom_list.append(
            1 / (magerr**2)
        ) 
    weighted_mean = sum(weighted_mean_num_list) / sum(weighted_mean_denom_list)
    return weighted_mean

#define weighted mean function for a list of list of mag_err
def get_weighted_mean_list(mag_list_list, mag_err_list_list):
    #list of weighted means
    weighted_mean_list = []
    for i, mag_list in enumerate(mag_list_list):
        weighted_mean_list.append(
            get_weighted_mean(mag_list, mag_err_list_list[i])
        )
        
    return weighted_mean_list

#chi^2 reduced, from HCV
def get_chi2_red_HCV(lightcurve_data):
    #extract matchid and filter_type from lightcurve_data
    matchid = lightcurve_data[0]
    filter_type = lightcurve_data[1]
    
    #get df for matchid and filter type
    filter_matchid_df = df.loc[(df['matchid']==matchid) & (df['filter']==filter_type)]
    
    #the HCV already calculated chi2, so pull the value
    chi2 = filter_matchid_df['chi2'].values[0]
    return chi2

#now calculate chi^2 reduced based on the MagErr parameter of big_df for comparison
def get_chi2_red(lightcurve_data):
    #extract matchid and filter_type from lightcurve_data
    matchid = lightcurve_data[0]
    filter_type = lightcurve_data[1]
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get list of magnitude errors
    mag_err_list = lightcurve_data[3]
    
    #chi2-------
    #first compute weighted mean
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    
    #now chi2
    chi2_list = []
    for i, mag in enumerate(mag_list):
        chi2_list.append(
            ((mag - weighted_mean)**2) / (mag_err_list[i]**2)
        )
        
    chi2 = sum(chi2_list)
    
    #chi2 reduced-----
    chi2_red = chi2 / (len(mag_list) - 1)
    return chi2_red

#weighted standard dev
def get_weighted_stdev(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get list of magnitude errors
    mag_err_list = lightcurve_data[3]
    
    #get weighted mean
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    
    #calculate first term multiplying the sum of the weights*(mag - magerr)^2
    weights_list = []
    square_weights_list = []
    for magerr in mag_err_list:
        weight = 1 / (magerr**2)
        weights_list.append(weight)
        square_weights_list.append(weight**2)
        
    left_term = sum(weights_list) / ((sum(weights_list)**2) - sum(square_weights_list))
    
    right_term_list = []
    for i, mag in enumerate(mag_list):
        right_term_list.append(
            weights_list[i]*((mag - weighted_mean)**2)
        )
        
    right_term = sum(right_term_list)
    
    #return square root of left term * right term
    weighted_std_dev = (left_term * right_term)**0.5
    return weighted_std_dev


#IQR
def get_iqr(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    
    #use np.percentile to find the value of 25% and 75% in the mag list
    q3, q1 = np.percentile(mag_list, [75, 25])
    iqr = q3 - q1
    return iqr

#robust median statistic
#for non-variable object, expected value is around 1
def get_roms(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get mag err
    mag_err_list = lightcurve_data[3]
    
    median_mag = median(mag_list)
    right_term_list = []
    for i, mag in enumerate(mag_list):
        right_term_list.append(
        abs(mag - median_mag) / mag_err_list[i]
        )
        
    right_term = sum(right_term_list)
    left_term = 1 / (len(mag_list) - 1)
    roms = left_term * right_term
    return roms

#normalized excess variance
def get_norm_excess_var(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get mag err
    mag_err_list = lightcurve_data[3]
    
    #get weighted mean
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    
    right_term_list = []
    for i, mag in enumerate(mag_list):
        right_term_list.append(
            ((mag - weighted_mean)**2 - (mag_err_list[i]**2))
        )
        
    right_term = sum(right_term_list)
    left_term = 1 / (len(mag_list)*(weighted_mean**2))
    norm_excess_var = left_term * right_term
    return norm_excess_var

#peak to peak variability
def get_peak_peak_var(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get mag err
    mag_err_list = lightcurve_data[3]
    
    #create list of mag + magerr and mag - magerr
    mag_plus_magerr_list = []
    mag_minus_magerr_list = []
    for i, mag in enumerate(mag_list):
        mag_plus_magerr_list.append(mag + mag_err_list[i])
        mag_minus_magerr_list.append(mag - mag_err_list[i])
        
    #find max and min for minus, plus respectively
    min_mag_plus_magerr = min(mag_plus_magerr_list)
    max_mag_minus_magerr = max(mag_minus_magerr_list)
    
    peak_to_peak_var = (max_mag_minus_magerr - min_mag_plus_magerr) / (max_mag_minus_magerr + min_mag_plus_magerr)
    return peak_to_peak_var
        
#Lag-1 autocorrection
#may not be of that much use to us since measurements not evenly spaced
def get_lag1_auto(lightcurve_data):
    #unzip time, mag tuples to compute stdev on mag list
    time_list, mag_list = zip(*lightcurve_data[2])
    #get mag err
    mag_err_list = lightcurve_data[3]
    
    #get weighted mean
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    
    num_list = []
    for i in range(len(mag_list)-1):
        num_list.append(
            (mag_list[i] - weighted_mean) * (mag_list[i+1] - weighted_mean)
        )
    
    num = sum(num_list)
        
    denom_list = []
    for mag in mag_list:
        denom_list.append(
            (mag - weighted_mean)**2
        )
        
    denom = sum(denom_list)
    
    lag1_auto = num / denom
    return lag1_auto

#Welch-Stetson variability index (I)
#need to take all data for all filters
def get_I(lightcurve_data_all, max_T):
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        j_paired_filters = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
            j_paired_filters.append((filters[i], filters[i+1]))
            
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        j_paired_filters.append((filters[-1], filters[0]))
        
        #for each filter pair, calculate I
        I_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)
            
            #get magerrs for each filter
            f1_mag_err_list = lightcurve_data_all[f1_index][3]
            f2_mag_err_list = lightcurve_data_all[f2_index][3]
            
            #compute mean mags for each filter
            f1_mean_mag = get_weighted_mean(f1_mag_list, f1_mag_err_list)
            f2_mean_mag = get_weighted_mean(f2_mag_list, f2_mag_err_list)

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
            else:
                max_index = len(f1_mag_list)
                
            #find pairs based on T
            time_mag_magerr_pair_list = []
            
            i = 0
            while i < max_index:
                #if difference exceeds, then append pair to individual lists
                if abs(f1_time_list[i] - f2_time_list[i]) <= max_T:
                    #save non-pair data to individual lists
                    time_mag_magerr_pair_list.append([(f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]), 
                                                     (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                                                     ])
                   
                i+=1
                    
            #calculate right term
            right_term_list = []
            for i in range(len(time_mag_magerr_pair_list)):
                right_term_list.append(
                    ((time_mag_magerr_pair_list[i][0][1] - f1_mean_mag) / time_mag_magerr_pair_list[i][0][2]) * ((time_mag_magerr_pair_list[i][1][1] - f2_mean_mag) / time_mag_magerr_pair_list[i][1][2])
                )
            #print(time_mag_magerr_pair_list)
            #print('f1', f1_time_list)
            #print('f2', f2_time_list)
            
            #check that the pair lists has pairs
            if len(time_mag_magerr_pair_list) > 1:
                right_term = sum(right_term_list)
                left_term = (1 / (len(time_mag_magerr_pair_list)*(len(time_mag_magerr_pair_list) - 1)))**0.5

                I = left_term * right_term
                
            else:
                I = 0
            
            I_list.append(I)
        #return median I
        return I_list, j_paired_filters
    
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        filter_type = lightcurve_data_all[0][1]
        time_list, mag_list = zip(*lightcurve_data_all[0][2])
        mag_err_list = lightcurve_data_all[0][3]
        
        #compute mean mag
        mean_mag = get_weighted_mean(mag_list, mag_err_list)
        
        #define the max time difference between pairs (days)
        max_T = max_T
        
        #create list of tuples (time, mag, magerr) to sort by time (0th index)
        time_mag_magerr_list = list(zip(time_list, mag_list, mag_err_list))
        
        #print(time_mag_magerr_list)
        
        #check if list has even or odd length; if odd, discard final measurement
        if len(time_mag_magerr_list) % 2 != 0:
            time_mag_magerr_list.pop(-1)
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list > 3
            if (len(even_list) <= 3) or (len(odd_list) <= 3):
                break
            
            #if not, then remove the pair
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index -= 1
                
            else:
                i+=1
                
        #now we can calculate I!
        right_term_list = []
        for i in range(len(even_list)):
            right_term_list.append(
                ((even_list[i][1] - mean_mag) / even_list[i][2]) * ((odd_list[i][1] - mean_mag) / odd_list[i][2])
            )
            
        right_term = sum(right_term_list)
        left_term = (1 / (len(even_list)*(len(even_list)-1)))**0.5
        I_list = [left_term * right_term]
        return I_list, [(filter_type, '')]
                
                
#helper functions for Stetson's J, K, and L
#iterate weighted mean -- gave errors due to non-convergence so not using this; see get_weighted_mean_list function instead
'''
def iterate_weighted_mean(mag_list_list, mag_err_list_list, weighted_mean_list):  
    #get next weighted means
    new_mag_err_list_list = []
    for i, mag_err_list in enumerate(mag_err_list_list):
        new_mag_err_list = []
        for j, mag_err in enumerate(mag_err_list):
            #multiplying factor
            f_num_left = (len(mag_list_list[i]) / (len(mag_list_list[i]) -1))**0.5
            f_num_right = ((mag_list_list[i][j] - weighted_mean_list[i]) / mag_err)
            f = (1 + (abs(f_num_left * f_num_right) / 2)**2)**(-1)
            new_mag_err_list.append(mag_err*f)
        new_mag_err_list_list.append(new_mag_err_list)
        
    
    #now calculate new weighted mean
    new_weighted_mean_list = []
    for i, mag_list in enumerate(mag_list_list):
        new_weighted_mean_list.append(get_weighted_mean(mag_list, new_mag_err_list_list[i]))
    
    return new_mag_err_list_list, new_weighted_mean_list

#is difference condition met?
def is_diff_met(diff_list, e):   
    for diff in diff_list:
        if abs(diff) > e:
            return False         
    return True

def get_weighted_mean_iter(mag_list_list, mag_err_list_list):
    #now calculate weighted means using an iterative process
    weighted_mean_list_initial = []
    for i, mag_list in enumerate(mag_list_list):
        weighted_mean_list_initial.append(get_weighted_mean(mag_list, mag_err_list_list[i]))

    #define sensitivity parameter that indicates convergence
    e = 0.1

    #print(mag_list_list)
    #print()
    #print(mag_err_list_list)

    new_mag_err_list_list, new_weighted_mean_list = iterate_weighted_mean(mag_list_list, mag_err_list_list, weighted_mean_list_initial)
    #print(new_weighted_mean_list)
    diff_list = list(np.subtract(new_weighted_mean_list, weighted_mean_list_initial))

    #need every element in diff list to be smaller in abs than e
    diff_met = is_diff_met(diff_list, e)
    #print(diff_list)

    while diff_met == False:
        #get latest mag err list list and weights
        new_new_mag_err_list_list, new_new_weighted_mean_list = iterate_weighted_mean(mag_list_list, new_mag_err_list_list, new_weighted_mean_list)
        new_diff_list = list(np.subtract(new_new_weighted_mean_list, new_weighted_mean_list))
        diff_met = is_diff_met(new_diff_list, e)

        #set new_new equal to new
        new_mag_err_list_list = new_new_mag_err_list_list 
        new_weighted_mean_list = new_new_weighted_mean_list

    return new_weighted_mean_list 
'''        

#sign function
def sgn(x):
    if x > 0:
        return 1
    else:
        return -1  

    
#flux-independent I
def get_Ifi(lightcurve_data_all, max_T):
    #store lists of mag, magerr to calculate weighted mean per filter
    time_list_list = []
    mag_list_list = []
    mag_err_list_list = []
    for i in range(len(lightcurve_data_all)):
        #extract lists
        time_list, mag_list = zip(*lightcurve_data_all[i][2])
        mag_err_list = lightcurve_data_all[i][3]
        #append to lists of lists
        time_list_list.append(time_list)
        mag_list_list.append(mag_list)
        mag_err_list_list.append(mag_err_list)

    #get iterated weighted mean list
    weighted_mean_list = get_weighted_mean_list(mag_list_list, mag_err_list_list)
    
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #need to split observations across filters into k groups of 2 quasi-simultaneous measurements
        #the assigned weight wk will be the median of the weights of each of the measurements
        
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        j_paired_filters = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
            j_paired_filters.append((filters[i], filters[i+1]))
        
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        j_paired_filters.append((filters[-1], filters[0]))
        
        Ifi_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)
            
            #get magerrs for each filter
            f1_mag_err_list = lightcurve_data_all[f1_index][3]
            f2_mag_err_list = lightcurve_data_all[f2_index][3]
            
            
            #compute mean mags for each filter
            f1_mean_mag = weighted_mean_list[f1_index]
            f2_mean_mag = weighted_mean_list[f2_index]
            
            #create list of tuples to hold leftover individual pairs
            time_mag_magerr_individual_list = []

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
                
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
                
            else:
                max_index = len(f1_mag_list)
                
            #need to confirm that each pair occurs within the max_T; else, consider pair individually
    
            time_mag_magerr_pair_list = []
            
            i = 0
            while i < max_index:
                #if difference exceeds, then append pair to individual lists
                if abs(f1_time_list[i] - f2_time_list[i]) <= max_T:
                    time_mag_magerr_pair_list.append([(f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]), 
                                                     (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                                                     ])     
                i+=1
                    
            #calculate right term
            right_term_list = []
            
            for i in range(len(time_mag_magerr_pair_list)):
                right_term_list.append(
                    sgn((time_mag_magerr_pair_list[i][0][1] - f1_mean_mag) / time_mag_magerr_pair_list[i][0][2]) * sgn((time_mag_magerr_pair_list[i][1][1] - f2_mean_mag) / time_mag_magerr_pair_list[i][1][2])
                )

            if len(time_mag_magerr_pair_list) > 1:
                right_term = sum(right_term_list)
            else:
                right_term = 0
            Ifi = ((1 / max_index) * right_term + 1)*0.5
            Ifi_list.append(Ifi)
        #return Ifi list
        return Ifi_list, j_paired_filters
    
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        filter_type = lightcurve_data_all[0][1]
        time_list, mag_list = zip(*lightcurve_data_all[0][2])
        mag_err_list = lightcurve_data_all[0][3]
        
        #compute mean mag
        mean_mag = get_weighted_mean(mag_list, mag_err_list)
        
        #define the max time difference between pairs (days)
        max_T = max_T
        
        #create list of tuples (time, mag, magerr) to sort by time (0th index)
        time_mag_magerr_list = list(zip(time_list, mag_list, mag_err_list))
        
        #print(time_mag_magerr_list)
        
        #check if list has even or odd length; if odd, discard final measurement
        if len(time_mag_magerr_list) % 2 != 0:
            time_mag_magerr_list.pop(-1)
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list > 3
            if (len(even_list) <= 3) or (len(odd_list) <= 3):
                break
            
            #if not, then remove the pair
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index -= 1
                
            else:
                i+=1
                
        #now we can calculate I!
        right_term_list = []
        for i in range(len(even_list)):
            right_term_list.append(
                sgn((even_list[i][1] - mean_mag) / even_list[i][2]) * sgn((odd_list[i][1] - mean_mag) / odd_list[i][2])
            )
            
        right_term = sum(right_term_list)
        
        Ifi_list = [(1 / (len(even_list))* right_term + 1) *0.5]
        return Ifi_list, [(filter_type, '')]    
    

#Stetson's J
def get_J(lightcurve_data_all, max_T):
    #store lists of mag, magerr to calculate weighted mean per filter
    time_list_list = []
    mag_list_list = []
    mag_err_list_list = []
    for i in range(len(lightcurve_data_all)):
        #extract lists
        time_list, mag_list = zip(*lightcurve_data_all[i][2])
        mag_err_list = lightcurve_data_all[i][3]
        #append to lists of lists
        time_list_list.append(time_list)
        mag_list_list.append(mag_list)
        mag_err_list_list.append(mag_err_list)

    #get iterated weighted mean list
    weighted_mean_list = get_weighted_mean_list(mag_list_list, mag_err_list_list)
    
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #need to split observations across filters into k groups of 2 quasi-simultaneous measurements
        #the assigned weight wk will be the median of the weights of each of the measurements
        
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
            
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        
        for pair in filter_pairs:
            f1_index = pair[1]
            f2_index = pair[2]
            
            f1_mag_err_list = mag_err_list_list[f1_index]
            f2_mag_err_list = mag_err_list_list[f2_index]
            
            #print('mag err 1', len(f1_mag_err_list))
            #print('mag err 2', len(f2_mag_err_list))
        
        #for each filter pair, calculate Pk
        J_num_list = []
        J_denom_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            
            f1_mag_err_list = mag_err_list_list[f1_index]
            f2_mag_err_list = mag_err_list_list[f2_index]
            
            #print('mag err 1 inside', f1_index, len(f1_mag_err_list))
            #print('mag err 2 inside', f2_index, len(f2_mag_err_list))
            
            
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)           
            
            
            #compute mean mags for each filter
            f1_mean_mag = weighted_mean_list[f1_index]
            f2_mean_mag = weighted_mean_list[f2_index]
            
            #print('f1 mag tuple', len(f1_mag_tuple))
            #print('f1 mag error', len(f1_mag_err_list))
            #print('f2 mag tuple', len(f2_mag_tuple))
            #print('f2 mag error', len(f2_mag_err_list))
            
            #create list of tuples to hold leftover individual pairs
            time_mag_magerr_individual_list = []

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f2_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                    )
                
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f1_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i])
                    )
                
            else:
                max_index = len(f1_mag_list)
                
            #need to confirm that each pair occurs within the max_T; else, consider pair individually
            
            time_mag_magerr_pair_list = []
            
            i = 0
            while i < max_index:
                #if difference exceeds, then append pair to individual lists
                if abs(f1_time_list[i] - f2_time_list[i]) > max_T:
                    #save non-pair data to individual lists
                    time_mag_magerr_individual_list.append(
                      (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]) 
                    )
                    
                    time_mag_magerr_individual_list.append(
                      (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i]) 
                    )   
                    
                #else, append to pair list, iterate to next
                else:
                    time_mag_magerr_pair_list.append([(f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]), 
                                                     (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                                                     ])
                i+=1
            
            #numerator calculation-----
            #first for all pairs
            #check if >1 pair
            if len(time_mag_magerr_pair_list) > 1:
            
                #get J num and denom for pairs
                for i in range(len(time_mag_magerr_pair_list)):
                    Pk_left_left = ((len(f1_mag_list)) / (len(f1_mag_list) - 1))**0.5
                    Pk_left_right = ((time_mag_magerr_pair_list[i][0][1] - f1_mean_mag) / time_mag_magerr_pair_list[i][0][2])
                    Pk_right_left = ((len(f2_mag_list)) / (len(f2_mag_list) - 1))**0.5
                    Pk_right_right = ((time_mag_magerr_pair_list[i][1][1] - f2_mean_mag) / time_mag_magerr_pair_list[i][1][2])
                    Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right

                    #get weight: median of individual weights
                    weight = median([1 / (time_mag_magerr_pair_list[i][0][2]**2), 1 / (time_mag_magerr_pair_list[i][1][2] **2)])

                    #append to num and denom lists
                    J_num_list.append(
                        weight * sgn(Pk) * (abs(Pk))**0.5
                    )

                    J_denom_list.append(
                        weight
                    )
            #check if there's something in the individual list
            if len(time_mag_magerr_individual_list) > 1:
                    
                #get J num and denom for individual
                for i, time_mag_magerr in enumerate(time_mag_magerr_individual_list):
                    #first consider the first filter----
                    Pk_left = len(time_mag_magerr_individual_list) / (len(time_mag_magerr_individual_list) - 1) 
                    Pk_right = ((time_mag_magerr[1] - f1_mean_mag) / time_mag_magerr[2])**2
                    Pk = Pk_left * Pk_right - 1

                    #weight: that for the filter
                    weight = 1 / (time_mag_magerr[2]**2)

                    #append to num and denom lists
                    J_num_list.append(
                        weight * sgn(Pk) * (abs(Pk))**0.5
                    )

                    J_denom_list.append(
                        weight
                    )
        
        
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        #print(lightcurve_data_all[0][2])
        
        #before sorting, create list of tuples
        time_mag_magerr_list = list(zip(time_list_list[0], mag_list_list[0], mag_err_list_list[0]))
        
        #check if list has even or odd length; if odd, add to individual measurement
        inidividual_time_mag_magerr_list = []
        if len(time_mag_magerr_list) % 2 != 0:
            inidividual_time_mag_magerr_list.append(time_mag_magerr_list[-1])
            time_mag_magerr_list.pop(-1)
            
        #get weighted mean
        mean = weighted_mean_list[0]
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list > 3
            if (len(even_list) <= 3) or (len(odd_list) <= 3):
                break
            
            #if not, then remove the pair from pair list but add to individual list
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                inidividual_time_mag_magerr_list.append(even_list[i])
                inidividual_time_mag_magerr_list.append(odd_list[i])
                
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index -= 1
                
            else:
                i+=1
                
        #now we can calculate J!
        J_num_list = []
        J_denom_list = []
        #first consider the pairs---
        for i in range(len(even_list)):
            Pk_left_left = ((len(even_list)) / (len(even_list) - 1))**0.5
            Pk_left_right = ((even_list[i][1] - mean) / even_list[i][2])
            Pk_right_left = ((len(odd_list)) / (len(odd_list) - 1))**0.5
            Pk_right_right = ((odd_list[i][1] - mean) / odd_list[i][2])
            Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right

            #get weight: median of individual weights
            weight = median([1 / (even_list[i][2]**2), 1 / (odd_list[i][2] **2)])

            #append to num and denom lists
            J_num_list.append(
                weight * sgn(Pk) * (abs(Pk))**0.5
            )

            J_denom_list.append(
                weight
            )
        #now consider the inidividual measurements
        if len(inidividual_time_mag_magerr_list) > 1:
            for i in range(len(inidividual_time_mag_magerr_list)):
                Pk_left = len(inidividual_time_mag_magerr_list) / (len(inidividual_time_mag_magerr_list) - 1) 
                Pk_right = ((inidividual_time_mag_magerr_list[i][1] - mean) / inidividual_time_mag_magerr_list[i][2])**2
                Pk = Pk_left * Pk_right - 1

                #weight: that for first filter
                weight = 1 / (inidividual_time_mag_magerr_list[i][2]**2)

                #append to num and denom lists
                J_num_list.append(
                    weight * sgn(Pk) * (abs(Pk))**0.5
                )

                J_denom_list.append(
                    weight
                )


    #now get complete J_num and J_denom
    J_num = sum(J_num_list)
    J_denom = sum(J_denom_list)
    if J_denom != 0:
        J = J_num / J_denom
    else:
        J = 0
    return J 
        
#Stetson's K
def get_K(lightcurve_data_all):
    mag_list_list = []
    mag_err_list_list = []
    for i in range(len(lightcurve_data_all)):
        #extract lists
        time_list, mag_list = zip(*lightcurve_data_all[i][2])
        mag_err_list = lightcurve_data_all[i][3]
        #append to lists of lists
        mag_list_list.append(mag_list)
        mag_err_list_list.append(mag_err_list)

    #get iterated weighted mean list
    weighted_mean_list = get_weighted_mean_list(mag_list_list, mag_err_list_list)
    
    #compute K!
    K_num_list = []  
    K_denom_list = []
    #total number of observations:
    N = 0
    for i, mag_list in enumerate(mag_list_list):
        for j, mag in enumerate(mag_list):
            left_term = (len(mag_list) / (len(mag_list) - 1))**0.5
            right_term = (mag - weighted_mean_list[i]) / mag_err_list_list[i][j]
            #append to K_num_list
            K_num_list.append(
                abs(left_term * right_term)
            )
            #append to K_denom_list
            K_denom_list.append(
                (left_term * right_term)**2
            )
        N+=len(mag_list)
    K_complete_num = (1 / N) * sum(K_num_list)
    K_complete_denom = ((1 / N) * sum(K_denom_list))**0.5
    K = K_complete_num / K_complete_denom
    return K
        
#Stetson's L!
def get_L(J, K):
    L = ((np.pi / 2)**0.5)*J*K
    return L

#Stetson's J but time weighting
#we use all of the above code for J, except we forgo a delta T parameter and instead weight using the time differneces
#i.e., w_i = exp(-(t_(i+1) - t_i) / delta t), where delta t is the median of all the pairs (t_(i+1) - t_i)
def get_J_time(lightcurve_data_all, max_T_nonpairs):
    #store lists of mag, magerr to calculate weighted mean per filter
    time_list_list = []
    mag_list_list = []
    mag_err_list_list = []
    for i in range(len(lightcurve_data_all)):
        #extract lists
        time_list, mag_list = zip(*lightcurve_data_all[i][2])
        mag_err_list = lightcurve_data_all[i][3]
        #append to lists of lists
        time_list_list.append(time_list)
        mag_list_list.append(mag_list)
        mag_err_list_list.append(mag_err_list)

    #get iterated weighted mean list
    weighted_mean_list = get_weighted_mean_list(mag_list_list, mag_err_list_list)
    
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #need to split observations across filters into k groups of 2 quasi-simultaneous measurements
        #the assigned weight wk will be the median of the weights of each of the measurements
        
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
        
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        
        #for each filter pair, calculate Pk
        J_num_list = []
        J_denom_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)
            
            #get magerrs for each filter
            f1_mag_err_list = lightcurve_data_all[f1_index][3]
            f2_mag_err_list = lightcurve_data_all[f2_index][3]
            
            #compute mean mags for each filter
            f1_mean_mag = weighted_mean_list[f1_index]
            f2_mean_mag = weighted_mean_list[f2_index]
            
            #create list of tuples to hold leftover individual pairs
            time_mag_magerr_individual_list = []

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f2_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                    )
                
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f1_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i])
                    )
                
            else:
                max_index = len(f1_mag_list)
                
            #intead of checking delta_T here, we need to calculate new weights
            #get list of the difference in time of observations
            time_diff = []
            for i in range(max_index):
                time_diff.append(abs(f1_time_list[i] - f2_time_list[i]))
            
            #find median
            median_time = median(time_diff)
            
            #now can compute the weights
            time_weights = []
            for diff in time_diff:
                time_weights.append(
                    np.exp(-(diff / median_time))
                )
                
            #numerator calculation-----
            #first for all pairs
            
            #get J num and denom for pairs
            for i in range(max_index):
                Pk_left_left = ((len(f1_mag_list)) / (len(f1_mag_list) - 1))**0.5
                Pk_left_right = ((f1_mag_list[i] - f1_mean_mag) / f1_mag_err_list[i])
                Pk_right_left = ((len(f2_mag_list)) / (len(f2_mag_list) - 1))**0.5
                Pk_right_right = ((f2_mag_list[i] - f2_mean_mag) / f2_mag_err_list[i])
                Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right
                
                #get weight: median of individual weights
                weight = time_weights[i]
            
                #append to num and denom lists
                J_num_list.append(
                    weight * sgn(Pk) * (abs(Pk))**0.5
                )
                
                J_denom_list.append(
                    weight
                )
                
            #get J num and denom for individual
            if len(time_mag_magerr_individual_list) > 1:
                for i, time_mag_magerr in enumerate(time_mag_magerr_individual_list):
                    #first consider the first filter----
                    Pk_left = len(time_mag_magerr_individual_list) / (len(time_mag_magerr_individual_list) - 1) 
                    Pk_right = ((time_mag_magerr[1] - f1_mean_mag) / time_mag_magerr[2])**2
                    Pk = Pk_left * Pk_right - 1

                    #weight: that for the filter
                    weight = 1 / (time_mag_magerr[2]**2)

                    #append to num and denom lists
                    J_num_list.append(
                        weight * sgn(Pk) * (abs(Pk))**0.5
                    )

                    J_denom_list.append(
                        weight
                    )
        
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        #print(lightcurve_data_all[0][2])
        
        #still need to set max_T for non-pairs
        max_T = max_T_nonpairs
        
        #before sorting, create list of tuples
        time_mag_magerr_list = list(zip(time_list_list[0], mag_list_list[0], mag_err_list_list[0]))
        
        #check if list has even or odd length; if odd, add to individual measurement
        inidividual_time_mag_magerr_list = []
        if len(time_mag_magerr_list) % 2 != 0:
            inidividual_time_mag_magerr_list.append(time_mag_magerr_list[-1])
            time_mag_magerr_list.pop(-1)
            
        #get weighted mean
        mean = weighted_mean_list[0]
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list >= 3
            if (len(even_list) < 3) or (len(odd_list) < 3):
                break
            
            #if not, then remove the pair from pair list but add to individual list
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                inidividual_time_mag_magerr_list.append(even_list[i])
                inidividual_time_mag_magerr_list.append(odd_list[i])
                
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index-=1
                
            else:
                i+=1
                
        #now we can calculate J!
        J_num_list = []
        J_denom_list = []
        #first consider the pairs---
        for i in range(len(even_list)):
            Pk_left_left = ((len(even_list)) / (len(even_list) - 1))**0.5
            Pk_left_right = ((even_list[i][1] - mean) / even_list[i][2])
            Pk_right_left = ((len(odd_list)) / (len(odd_list) - 1))**0.5
            Pk_right_right = ((odd_list[i][1] - mean) / odd_list[i][2])
            Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right

            #get weight: median of individual weights
            weight = median([1 / (even_list[i][2]**2), 1 / (odd_list[i][2] **2)])

            #append to num and denom lists
            J_num_list.append(
                weight * sgn(Pk) * (abs(Pk))**0.5
            )

            J_denom_list.append(
                weight
            )
        #now consider the inidividual measurements
        if len(inidividual_time_mag_magerr_list) > 1:
            for i in range(len(inidividual_time_mag_magerr_list)):
                Pk_left = len(inidividual_time_mag_magerr_list) / (len(inidividual_time_mag_magerr_list) - 1) 
                Pk_right = ((inidividual_time_mag_magerr_list[i][1] - mean) / inidividual_time_mag_magerr_list[i][2])**2
                Pk = Pk_left * Pk_right - 1

                #weight: that for first filter
                weight = 1 / (inidividual_time_mag_magerr_list[i][2]**2)

                #append to num and denom lists
                J_num_list.append(
                    weight * sgn(Pk) * (abs(Pk))**0.5
                )

                J_denom_list.append(
                    weight
                )
            

    #now get complete J_num and J_denom
    J_num = sum(J_num_list)
    J_denom = sum(J_denom_list)
    if J_denom != 0:
            J = J_num / J_denom
    else:
        J = 0
    return J 
    
#I clipped
#like I, except we impose in addition to the time constraint that the pairs' mag must not differ by > 5x combined uncertainty
def get_I_clipped(lightcurve_data_all, max_T):
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        j_paired_filters = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
            j_paired_filters.append((filters[i], filters[i+1]))
            
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        j_paired_filters.append((filters[-1], filters[0]))
        
        #for each filter pair, calculate I
        I_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)
            
            #get magerrs for each filter
            f1_mag_err_list = lightcurve_data_all[f1_index][3]
            f2_mag_err_list = lightcurve_data_all[f2_index][3]
            
            #compute mean mags for each filter
            f1_mean_mag = get_weighted_mean(f1_mag_list, f1_mag_err_list)
            f2_mean_mag = get_weighted_mean(f2_mag_list, f2_mag_err_list)

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
            else:
                max_index = len(f1_mag_list)
                
            #need to confirm that each pair occurs within the max_T; else, remove pair
            time_mag_magerr_pair_list = []
            i = 0
            while i < max_index:
                
                #if difference exceeds, then remove pair
                if (abs(f1_time_list[i] - f2_time_list[i]) <= max_T) and (abs(f1_mag_list[i] - f2_mag_list[i]) <= 5*(f1_mag_err_list[i] + f2_mag_err_list[i])):
                    time_mag_magerr_pair_list.append([
                        (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]), 
                        (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                    ])
                
                i+=1
                    
            #calculate right term
            right_term_list = []
            if len(time_mag_magerr_pair_list) > 1:
                for i in range(len(time_mag_magerr_pair_list)):
                    right_term_list.append(
                        ((time_mag_magerr_pair_list[i][0][1] - f1_mean_mag) / time_mag_magerr_pair_list[i][0][2]) * ((time_mag_magerr_pair_list[i][1][1] - f2_mean_mag) / time_mag_magerr_pair_list[i][1][2])
                    )

                right_term = sum(right_term_list)
                left_term = (1 / (max_index*(max_index - 1)))**0.5

                I = left_term * right_term
            else:
                I = 0
            I_list.append(I)
        
        return I_list, j_paired_filters
    
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        filter_type = lightcurve_data_all[0][1]
        time_list, mag_list = zip(*lightcurve_data_all[0][2])
        mag_err_list = lightcurve_data_all[0][3]
        
        #compute mean mag
        mean_mag = get_weighted_mean(mag_list, mag_err_list)
        
        #define the max time difference between pairs (days)
        max_T = max_T
        
        #create list of tuples (time, mag, magerr) to sort by time (0th index)
        time_mag_magerr_list = list(zip(time_list, mag_list, mag_err_list))
        
        #print(time_mag_magerr_list)
        
        #check if list has even or odd length; if odd, discard final measurement
        if len(time_mag_magerr_list) % 2 != 0:
            time_mag_magerr_list.pop(-1)
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list > 3
            if (len(even_list) <= 3) or (len(odd_list) <= 3):
                break
            
            #if not, then remove the pair
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index =- 1
            
            #now check for magnitude uncertainty
            elif abs(even_list[i][1] - odd_list[i][1]) > 5*(even_list[i][2] + odd_list[i][2]):
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index =- 1
            
            else:
                i+=1
                
        #now we can calculate I!
        right_term_list = []
        for i in range(len(even_list)):
            right_term_list.append(
                ((even_list[i][1] - mean_mag) / even_list[i][2]) * ((odd_list[i][1] - mean_mag) / odd_list[i][2])
            )
            
        right_term = sum(right_term_list)
        left_term = (1 / (len(even_list)*(len(even_list)-1)))**0.5
        I = [left_term * right_term]
        return I, [(filter_type, '')]

#clipped J
#same as J, but with same delta mag restriction as with I clipped
def get_J_clipped(lightcurve_data_all, max_T):
    #store lists of mag, magerr to calculate weighted mean per filter
    time_list_list = []
    mag_list_list = []
    mag_err_list_list = []
    for i in range(len(lightcurve_data_all)):
        #extract lists
        time_list, mag_list = zip(*lightcurve_data_all[i][2])
        mag_err_list = lightcurve_data_all[i][3]
        #append to lists of lists
        time_list_list.append(time_list)
        mag_list_list.append(mag_list)
        mag_err_list_list.append(mag_err_list)

    #get iterated weighted mean list
    weighted_mean_list = get_weighted_mean_list(mag_list_list, mag_err_list_list)
    
    #figure out how many filters
    if len(lightcurve_data_all) > 1:
        #need to split observations across filters into k groups of 2 quasi-simultaneous measurements
        #the assigned weight wk will be the median of the weights of each of the measurements
        
        #extract all the filters
        filters = []
        for i in range(len(lightcurve_data_all)):
            filters.append(lightcurve_data_all[i][1])
        
        #create pairs of filters along with indices
        filter_pairs = []
        for i in range(len(filters)-1):
            filter_pairs.append(
                [(filters[i], filters[i+1]), i, i+1]
            )
        
        #pair final with initial
        filter_pairs.append(
            [(filters[-1], filters[0]), -1, 0]
        )
        
        #for each filter pair, calculate Pk
        J_num_list = []
        J_denom_list = []
        for pair in filter_pairs:
            #get lists of mag and magerr per filter
            f1_index = pair[1]
            f2_index = pair[2]
            f1_time_tuple, f1_mag_tuple = zip(*lightcurve_data_all[f1_index][2])
            f2_time_tuple, f2_mag_tuple = zip(*lightcurve_data_all[f2_index][2])
            #convert to lists
            f1_time_list = list(f1_time_tuple)
            f2_time_list = list(f2_time_tuple)
            f1_mag_list = list(f1_mag_tuple)
            f2_mag_list = list(f2_mag_tuple)
            
            #get magerrs for each filter
            f1_mag_err_list = lightcurve_data_all[f1_index][3]
            f2_mag_err_list = lightcurve_data_all[f2_index][3]
            
            #compute mean mags for each filter
            f1_mean_mag = weighted_mean_list[f1_index]
            f2_mean_mag = weighted_mean_list[f2_index]
            
            #create list of tuples to hold leftover individual pairs
            time_mag_magerr_individual_list = []

            #compute the lengths of the two mag lists; index over whichever is shortest
            if len(f1_mag_list) < len(f2_mag_list):
                max_index = len(f1_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f2_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                    )
                
            elif len(f1_mag_list) > len(f2_mag_list):
                max_index = len(f2_mag_list)
                #save the extra measurements to the individual list
                for i in range(max_index, len(f1_mag_list)):
                    time_mag_magerr_individual_list.append(
                        (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i])
                    )
                
            else:
                max_index = len(f1_mag_list)
                
            #need to confirm that each pair occurs within the max_T; else, consider pair individually
            time_mag_magerr_pair_list = []
            i = 0
            while i < max_index:
                #if difference exceeds, then append pair to individual lists
                #check to make sure mag diff does not exceed 5* combined uncertainty
                if (abs(f1_time_list[i] - f2_time_list[i]) <= max_T) and (abs(f1_mag_list[i] - f2_mag_list[i]) <= 5 * (f1_mag_err_list[i] + f2_mag_err_list[i])):
                    time_mag_magerr_pair_list.append([
                      (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]),
                      (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i])
                    ])

                
                else:
                    #save non-pair data to individual lists
                    time_mag_magerr_individual_list.append(
                      (f1_time_list[i], f1_mag_list[i], f1_mag_err_list[i]) 
                    )
                    
                    time_mag_magerr_individual_list.append(
                      (f2_time_list[i], f2_mag_list[i], f2_mag_err_list[i]) 
                    )
                    
                    
                i+=1

            #numerator calculation-----
            #first for all pairs
            
            #get J num and denom for pairs
            if len(time_mag_magerr_pair_list) > 1:
                for i in range(len(time_mag_magerr_pair_list)):
                    Pk_left_left = ((len(f1_mag_list)) / (len(f1_mag_list) - 1))**0.5
                    Pk_left_right = ((time_mag_magerr_pair_list[i][0][1] - f1_mean_mag) / time_mag_magerr_pair_list[i][0][2])
                    Pk_right_left = ((len(f2_mag_list)) / (len(f2_mag_list) - 1))**0.5
                    Pk_right_right = ((time_mag_magerr_pair_list[i][1][1] - f2_mean_mag) / time_mag_magerr_pair_list[i][1][2])
                    Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right

                    #get weight: median of individual weights
                    weight = median([1 / (time_mag_magerr_pair_list[i][0][2]**2), 1 / (time_mag_magerr_pair_list[i][1][2] **2)])

                    #append to num and denom lists
                    J_num_list.append(
                        weight * sgn(Pk) * (abs(Pk))**0.5
                    )

                    J_denom_list.append(
                        weight
                    )
                
            #print(time_mag_magerr_individual_list)
            
            #get J num and denom for individual
            if len(time_mag_magerr_individual_list) > 1:
                for i in range(len(time_mag_magerr_individual_list)):
                    
                    #first consider the first filter----
                    Pk_left = len(time_mag_magerr_individual_list) / (len(time_mag_magerr_individual_list) - 1) 
                    Pk_right = ((time_mag_magerr_individual_list[i][1] - f1_mean_mag) / time_mag_magerr_individual_list[i][2])**2
                    Pk = Pk_left * Pk_right - 1

                    #weight: that for the filter
                    weight = 1 / (time_mag_magerr_individual_list[i][2]**2)

                    #append to num and denom lists
                    J_num_list.append(
                        weight * sgn(Pk) * (abs(Pk))**0.5
                    )

                    J_denom_list.append(
                        weight
                    )
        
    else:
        #if only 1 filter, assign odd indices to one sample, even to another
        #but use f1_mean_mag = f2_mean_mag over all the N measurements
        #print(lightcurve_data_all[0][2])
        
        #before sorting, create list of tuples
        time_mag_magerr_list = list(zip(time_list_list[0], mag_list_list[0], mag_err_list_list[0]))
        
        #check if list has even or odd length; if odd, add to individual measurement
        inidividual_time_mag_magerr_list = []
        if len(time_mag_magerr_list) % 2 != 0:
            inidividual_time_mag_magerr_list.append(time_mag_magerr_list[-1])
            time_mag_magerr_list.pop(-1)
            
        #get weighted mean
        mean = weighted_mean_list[0]
        
        #now iterate through and append to even/odd lists by index
        even_list = []
        odd_list = []
        for i in range(len(time_mag_magerr_list)):
            #even
            if i%2 == 0:
                even_list.append(time_mag_magerr_list[i])
            else:
                odd_list.append(time_mag_magerr_list[i])
        
        #confirm time differences in measurements < max_T
        i = 0
        max_index = len(even_list)
        while i < max_index:
            #first check if the length of the list > 3
            if (len(even_list) <= 3) or (len(odd_list) <= 3):
                break
            
            #if not, then remove the pair from pair list but add to individual list
            elif abs(even_list[i][0] - odd_list[i][0]) > max_T:
                inidividual_time_mag_magerr_list.append(even_list[i])
                inidividual_time_mag_magerr_list.append(odd_list[i])
                
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index -= 1
                
            #check to confirm delta mag is within 5*combined uncertainty
            elif abs(even_list[i][1] - odd_list[i][1]) > 5*(even_list[i][2] + odd_list[i][2]):
                inidividual_time_mag_magerr_list.append(even_list[i])
                inidividual_time_mag_magerr_list.append(odd_list[i])
                
                even_list.pop(i)
                odd_list.pop(i)
                
                max_index -= 1
            
            else:
                i+=1
                
        #now we can calculate J!
        J_num_list = []
        J_denom_list = []
        #first consider the pairs---
        for i in range(len(even_list)):
            Pk_left_left = ((len(even_list)) / (len(even_list) - 1))**0.5
            Pk_left_right = ((even_list[i][1] - mean) / even_list[i][2])
            Pk_right_left = ((len(odd_list)) / (len(odd_list) - 1))**0.5
            Pk_right_right = ((odd_list[i][1] - mean) / odd_list[i][2])
            Pk = Pk_left_left * Pk_left_right * Pk_right_left * Pk_right_right

            #get weight: median of individual weights
            weight = median([1 / (even_list[i][2]**2), 1 / (odd_list[i][2] **2)])

            #append to num and denom lists
            J_num_list.append(
                weight * sgn(Pk) * (abs(Pk))**0.5
            )

            J_denom_list.append(
                weight
            )
        #now consider the inidividual measurements
        if len(inidividual_time_mag_magerr_list) > 1:
            for i in range(len(inidividual_time_mag_magerr_list)):
                Pk_left = len(inidividual_time_mag_magerr_list) / (len(inidividual_time_mag_magerr_list) - 1) 
                Pk_right = ((inidividual_time_mag_magerr_list[i][1] - mean) / inidividual_time_mag_magerr_list[i][2])**2
                Pk = Pk_left * Pk_right - 1

                #weight: that for first filter
                weight = 1 / (inidividual_time_mag_magerr_list[i][2]**2)

                #append to num and denom lists
                J_num_list.append(
                    weight * sgn(Pk) * (abs(Pk))**0.5
                )

                J_denom_list.append(
                    weight
            )
            

    #now get complete J_num and J_denom
    J_num = sum(J_num_list)
    J_denom = sum(J_denom_list)
    if J_denom != 0:
        J = J_num / J_denom
    else:
        J = 0
    return J 

#consecutive same-sign deviations from the median mag
#number of groups with 3 consecutive measurements brighter/fainter than median mag by >= 3*sigma,
#where sigma is MAD scaled to sigma (sigma = 1.4826 * MAD)
def get_CSSD(lightcurve_data, mad):
    #extract matchid, mag info from lightcurve data
    matchid = lightcurve_data[0]
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
    
    #compute median of mag_list
    median_mag = median(mag_list)
    
    #define sigma
    sigma = 1.4826 * mad
    
    #print('sigma', sigma)
    #print('3sigma', 3* sigma)
    
    #find number of consecutive groups with 3 measurements with abs(mag - median) >= 3*sigma
    #initialize CSSD
    CSSD = 0
    
    #iterate through each element in the mag list and compare difference to median
    i = 0
    #use a while loop so we can update the counter to move beyond elements we've already checked
    while i < len(mag_list):
        #if we have a significant deviation from median, use for loop to check remaining elements
        #print('mag diff', abs(mag_list[i] - median_mag))
        if abs(mag_list[i] - median_mag) > 3*sigma:
            #temp to hold how many significant deviations in this round
            N = 1
            #start the for loop from the next index
            for j in range(i+1, len(mag_list)):
                #print('for diff', abs(mag_list[j] - median_mag))
                
                #if this deviation is significant, increment N
                if abs(mag_list[j] - median_mag) > 3*sigma:
                    #print('suc', abs(mag_list[j] - median_mag))
                    N+=1
                #if we hit a non-significant deviation, increment i and break since CSSD must be consecutive
                else:
                   #update i by difference of indices
                    i+= (j-i)
                    #break out of this for loop, back into the while loop
                    break 
                
                #check if we've hit N=3
                if N==3:
                    #print('increment')
                    #increment CSSD by 1
                    CSSD +=1
                    #update i by difference of indices
                    i+= (j-i)
                    #break out of this for loop, back into the while loop
                    break
            #if we make it here, then we never hit N==3; update counter, which will then trigger us to exit the while loop
            i+= len(mag_list)
    
        #increment counter
        i+=1
    
    #need to normalize the number of groups by (N-2), where N is number of pts on lightcurve
    CSSD_normalized = CSSD / (len(mag_list) - 2)
    
    #return the value of CSSD_normalized
    return CSSD_normalized

#excursions
def get_Ex(lightcurve_data, max_T):
    #extract matchid, mag info from lightcurve data
    matchid = lightcurve_data[0]
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
    
    #now divide mag_list into groups based on max_T
    #define list to hold sub-lists of mag tuples based on time
    groups_list = []
    #define temp list to hold mag for individual group; start it with the first mag
    group = [mag_list[0]]

    for i in range(len(mag_list)-1):
        #if the time difference is within max allowed time, save the next element to the group
        if time_list[i+1] - time_list[i] <= max_T:
            group.append(mag_list[i+1])
            #need to check if we're at the end of the list; if so, save the group!
            if i == len(mag_list) - 2:
                groups_list.append(group)
        
        #if not, then don't append the next element; add group to groups_list, restart group with i+1 mag
        else:
            groups_list.append(group)
            group = [mag_list[i+1]]
            #if i+1 = len(mag_list) - 1, then save this final group
            if i == len(mag_list) -2:
                #print(mag_list[i+1])
                groups_list.append(group)
            
    #compute median for each group; find mad for each group to find sigma for each group
    group_medians = []
    group_sigma = []
    for group in groups_list:
        median_mag = median(group)
        group_medians.append(median_mag)
        
        #MAD calculation
        #find absolute deviations
        abs_dev = []
        for mag in group:
            abs_dev.append(
                abs(mag - median_mag)
            )
        mad = median(abs_dev)
        sigma = 1.4826 * mad
        group_sigma.append(sigma)
    
    #now can compute terms to sum
    Ex_right_list = []
    for i in range(len(groups_list)-1):
        num = abs(group_medians[i] - group_medians[i+1])
        denom = ((group_sigma[i]**2) + (group_sigma[i+1]**2))**0.5
        if denom != 0:
            Ex_right_list.append(num / denom)
        else:
            Ex_right_list.append(0)
    Ex_right_term = sum(Ex_right_list)
    
    #also get the left term
    if len(groups_list) > 1:
        Ex_left_term = 2 / (len(groups_list)*(len(groups_list) - 1))
        Ex = Ex_left_term * Ex_right_term
    else:
        Ex = 0
    return Ex
    
#von Neumann ratio: 1 / eta
def get_eta_ratio(lightcurve_data):
    #extract matchid, mag info from lightcurve data
    matchid = lightcurve_data[0]
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
    
    mag_err_list = lightcurve_data[3]
    
    #define numerator and denomimator lists over which to sum and then divide
    num_list = []
    denom_list = []
    
    #for num list
    for i in range(len(mag_list) - 1):
        num_list.append(
            ((mag_list[i+1] - mag_list[i])**2) / (len(mag_list) - 1)
        )
    num = sum(num_list)
    
    #for denom list
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    for i, mag in enumerate(mag_list):
        denom_list.append(
            ((mag - weighted_mean)**2) / (len(mag_list) - 1)
        )
    denom = sum(denom_list) 
    
    eta = num / denom
    #we want to return 1 / eta as the measure, so take the reciprocal
    if eta != 0:
        return 1 / eta
    else:
        return 999999999
    
#SB
def get_SB(lightcurve_data):
    #first need to find the groups of consecutive same sign residuals; so compute list of residuals
    
    #extract matchid, mag info from lightcurve data
    matchid = lightcurve_data[0]
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
    mag_err_list = lightcurve_data[3]
    #weighted mean
    weighted_mean = get_weighted_mean(mag_list, mag_err_list)
    
    residuals_list = []
    residuals_sign = []
    for i, mag in enumerate(mag_list):
        residual = mag - weighted_mean 
        residuals_list.append(residual)
        residuals_sign.append(sgn(residual)) 
        
    #list of consecutive same-sign residuals; each element is a list of (residual, error) tuples
    CSSRG_list = []
    #now go through residuals sign list and group based on sign
    i = 0 
    while i < len(residuals_sign):    
        #if we're at the end of the list, then break
        if i==len(residuals_sign)-1:
            break
        
        #if +1, check if other +1
        if residuals_sign[i]==1:
            #temp list holding the (residual, error) tuples
            CSSRG = [(residuals_list[i], mag_err_list[i])]
            for j in range(i+1, len(residuals_sign)):
                #if we run into a -1, end of consecutive group
                if residuals_sign[j] == -1:
                    #check to see there are at least 2 objects in group; otherwise, if only one object, can't be a group
                    if len(CSSRG)<2:
                        #don't append CSSRG to CSSRG_list; increment i; need to subract 1 so we start the next group at the dissenting index
                        i += (j-i)-1
                        break
                    #if we have >= 2 residuals in a group, but groups ends, append to CSSRG_list
                    else:
                        CSSRG_list.append(CSSRG)
                        i += (j-i)-1
                        break
                #same sign!
                else:
                    CSSRG.append((residuals_list[j], mag_err_list[j]))
                    #if i == max_index-1 and we haven't hit a -1, append CSSRG to main list
                    if j == len(residuals_sign)-1:
                        CSSRG_list.append(CSSRG)
        #if -1, check if other -1
        else:
            #temp list holding the (residual, error) tuples
            CSSRG = [(residuals_list[i], mag_err_list[i])]
            for j in range(i+1, len(residuals_sign)):
                #if we run into a +1, end of consecutive group
                if residuals_sign[j] == 1:
                    #check to see there are at least 2 objects in group; otherwise, if only one object, can't be a group
                    if len(CSSRG)<2:
                        #don't append CSSRG to CSSRG_list; increment i
                        i += (j-i)-1
                        break
                    #if we have >= 2 residuals in a group, but groups ends, append to CSSRG_list
                    else:
                        CSSRG_list.append(CSSRG)
                        i += (j-i)-1
                        break
                #same sign!
                else:
                    CSSRG.append((residuals_list[j], mag_err_list[j]))
                    #if i == max_index-1 and we haven't hit a -1, append CSSRG to main list
                    if j == len(residuals_sign)-1:
                        CSSRG_list.append(CSSRG)
        i+=1
    
    #with the list of list of (residual, error) tuples, we can actually compute SB!!
    right_term_list = []
    for resid_err_list in CSSRG_list:
        #define temp var to hold sum generated in second for loop
        resid_err_sum = 0
        for resid_err in resid_err_list:
            resid_err_sum += (resid_err[0] / resid_err[1])
        #now append the square of the sum to the right_term_list
        right_term_list.append(
            resid_err_sum**2
        )
    if (len(mag_list) > 1) and (len(CSSRG_list) > 1):
        right_term = sum(right_term_list)
        left_term = 1 / (len(mag_list)*len(CSSRG_list))
        SB = left_term * right_term
    else:
        SB = 0
    return SB
            

#clipped mag -- remove top brighest and dimmest sources from lightcurve
def get_clipped(mag_list):
    #put high mag first (i.e. dimmer)
    dim_ordered = sorted(mag_list, reverse=True)
    #order based on decreasing mag (i.e. brighter)
    bright_ordered = sorted(mag_list)
    
    if (len(mag_list) >= 5) and (len(mag_list) <= 10):
        #remove brightest and dimmest source
        mag_list.remove(dim_ordered[0])
        mag_list.remove(bright_ordered[0])
        
    elif (len(mag_list) > 10) and (len(mag_list) <= 15):
        #drop the first two from dim and bright lists
        for i in range(0, 2):
            mag_list.remove(dim_ordered[i])
            mag_list.remove(bright_ordered[i])
            
    elif (len(mag_list) > 15) and (len(mag_list) <= 20):
        #drop the first three from dim and bright lists
        for i in range(0, 3):
            mag_list.remove(dim_ordered[i])
            mag_list.remove(bright_ordered[i])
            
    elif (len(mag_list) > 20) and (len(mag_list) <= 25):
        #drop the first four from dim and bright lists
        for i in range(0, 4):
            mag_list.remove(dim_ordered[i])
            mag_list.remove(bright_ordered[i])
    
    else:
        #drop the first five from dim and bright lists
        for i in range(0, 5):
            mag_list.remove(dim_ordered[i])
            mag_list.remove(bright_ordered[i])
            
    return mag_list
            

#clipped stdev
def get_clipped_stdev(lightcurve_data):
    #extract mag and time lists
    time_list, mag_list = zip(*lightcurve_data[2])
    mag_list = list(mag_list)
   
    #get clipped mag_list
    mag_list = get_clipped(mag_list)
    mean_mag = mean(mag_list)
    
    sq_residuals = []
    for mag in mag_list:
        sq_residuals.append(
            (mag - mean_mag)**2
        )
    right_term = sum(sq_residuals)
    left_term = 1 / (len(mag_list)-1)
    stdev = (left_term * right_term)**0.5
    return stdev
    
    
    

### Build matrix for each matchid/filter combo and feed lightcurve data through each function

In [43]:
#define empty dataframe to hold Stetson indices
king_kong_data = {
    'matchid': [], 'filter': [],
    'mad': [], 'chi2red': [], 'weighted stdev': [], 'clipped stdev': [], 'iqr': [], 'roms': [], 'norm excess var':[],
    'peak to peak var': [], 'lag1 auto': [], 'cssd': [], 'Ex': [], 'eta ratio': [], 'SB': [],
    'pair1': [], 'pair2': [], 'I': [], 'Ifi': [], 'Iclipped': [], 'J': [], 'K': [], 'L': [], 'Jtime': [],
    'Ltime': [], 'Jclipped': [], 'Lclipped': []
}

king_kong_mega = pd.DataFrame(king_kong_data)
#test dataframe for matchid 352
king_kong_875 = pd.DataFrame(king_kong_data)
#dataframe for 13 candidates in SWEEPS-606/814
king_kong_candidates = pd.DataFrame(king_kong_data)

In [44]:
#master function to populate all indices; takes in dataframe and matchid
def populate_king_kong(variability_matrix, matchid):
    #get list of list of lightcurve data tuples per filter
    lightcurve_data_all = get_lightcurve_data(matchid)
    
    #print(matchid)
    #get unique filter list
    unique_filters = df.loc[df['matchid']==matchid]['filter'].unique()
    
    #first calculate the indices that take lightcurve_data_all-----
    #set max_T: defines max duration to consider pairs of points "quasi-simultaneous"
    #here, we have chosen the conservative 100 days
    max_T = 100
    I_list, I_filters = get_I(lightcurve_data_all, max_T)
    Ifi_list, Ifi_filters = get_Ifi(lightcurve_data_all, max_T)
    Iclipped_list, Iclipped_filters = get_I_clipped(lightcurve_data_all, max_T)
    J = get_J(lightcurve_data_all, max_T)
    K = get_K(lightcurve_data_all)
    L = get_L(J, K)
    Jtime = get_J_time(lightcurve_data_all, max_T)
    Ltime = get_L(Jtime, K)
    Jclipped = get_J_clipped(lightcurve_data_all, max_T)
    Lclipped = get_L(Jclipped, K)
    
    #now calculate indices that take only lightcurve_data-------
    #define lists to hold the indices for all filters
    mad_list = []
    chi2red_list = []
    weighted_stdev_list = []
    clipped_stdev_list = []
    iqr_list = []
    roms_list = []
    norm_excess_var_list = []
    peak_peak_var_list = []
    lag1_auto_list = []
    cssd_list = []
    Ex_list = []
    eta_ratio_list = []
    SB_list = []
    for lightcurve_data in lightcurve_data_all:
        mad_list.append(get_MAD(lightcurve_data))
        chi2red_list.append(get_chi2_red(lightcurve_data))
        weighted_stdev_list.append(get_weighted_stdev(lightcurve_data))
        clipped_stdev_list.append(get_clipped_stdev(lightcurve_data))
        iqr_list.append(get_iqr(lightcurve_data))
        roms_list.append(get_roms(lightcurve_data))
        norm_excess_var_list.append(get_norm_excess_var(lightcurve_data))
        peak_peak_var_list.append(get_peak_peak_var(lightcurve_data))
        lag1_auto_list.append(get_lag1_auto(lightcurve_data))
        cssd_list.append(get_CSSD(lightcurve_data, get_MAD(lightcurve_data)))
        Ex_list.append(get_Ex(lightcurve_data, max_T))
        eta_ratio_list.append(get_eta_ratio(lightcurve_data))
        SB_list.append(get_SB(lightcurve_data))
    
    #now append to dataframe
    for i in range(len(lightcurve_data_all)):
        variability_matrix = variability_matrix.append({
            'matchid': matchid, 'filter': unique_filters[i],
            'mad': mad_list[i], 'chi2red': chi2red_list[i], 'weighted stdev': weighted_stdev_list[i], 
            'clipped stdev': clipped_stdev_list[i], 'iqr': iqr_list[i], 'roms': roms_list[i], 
            'norm excess var':norm_excess_var_list[i],'peak to peak var': peak_peak_var_list[i], 
            'lag1 auto': lag1_auto_list[i], 'cssd': cssd_list[i], 'Ex': Ex_list[i], 
            'eta ratio': eta_ratio_list[i], 'SB': SB_list[i],
            'pair1': I_filters[i][0], 'pair2': I_filters[i][1], 'I': I_list[i], 
            'Ifi': Ifi_list[i], 
            'Iclipped': Iclipped_list[i], 'J': J, 'K': K, 'L': L, 'Jtime': Jtime,
            'Ltime': Ltime, 'Jclipped': Jclipped, 'Lclipped': Lclipped
        }, ignore_index=True)
        
    return variability_matrix
        

### Populate king kong variability matrix

In [35]:
#test for matchid 875
matchids_test = [12670, 352, 875]
for matchid in matchids_test:
    current_df = pd.DataFrame(king_kong_data)
    king_kong_875 = king_kong_875.append(populate_king_kong(current_df, matchid), ignore_index=True)
king_kong_875

,matchid,filter,mad,chi2red,weighted stdev,clipped stdev,iqr,roms,norm excess var,peak to peak var,...,I,Ifi,Iclipped,J,K,L,Jtime,Ltime,Jclipped,Lclipped
0,12670.0,WFC3_F275W,0.018301,0.711289,0.059782,0.017163,0.034300,0.622210,-0.000003,0.000248,...,1.929661,1.000000,0.000000,1.305159,0.609604,0.997174,0.265751,0.203040,26.495066,20.242892
1,12670.0,WFC3_F336W,0.033300,6.763223,0.082584,0.024255,0.047199,2.048502,0.000017,0.004331,...,9.919714,0.600000,0.000000,1.305159,0.609604,0.997174,0.265751,0.203040,26.495066,20.242892
2,12670.0,WFC3_F438W,0.082400,97.610786,0.291537,0.075475,0.141600,7.289359,0.000129,0.015210,...,3.607986,0.600000,0.000000,1.305159,0.609604,0.997174,0.265751,0.203040,26.495066,20.242892
3,352.0,ACS_F606W,0.118000,2.291321,0.174979,0.128560,0.330575,1.292976,0.000047,0.008823,...,-2.391036,0.409091,0.000000,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
4,352.0,ACS_F814W,0.327699,40.643958,0.303004,0.258211,0.582800,6.762626,0.000203,0.020910,...,-2.391036,0.409091,0.000000,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
5,875.0,ACS_F435W,0.038400,7.116879,0.051365,0.032218,0.071100,2.339009,0.000004,0.003341,...,18.374893,0.722222,0.000000,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
6,875.0,ACS_F606W,0.036298,91.499882,0.047964,0.043002,0.073601,8.784496,0.000005,0.003046,...,-11.192250,0.500000,0.000000,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
7,875.0,ACS_F814W,0.032501,141.717121,0.056348,0.024663,0.057800,8.107334,0.000009,0.007049,...,0.000000,0.500000,0.000000,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
8,875.0,WFC3_F105W,0.042100,201.369578,0.069975,0.035618,0.068676,11.287296,0.000009,0.005810,...,70.556374,0.666667,0.000000,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
9,875.0,WFC3_F125W,0.071199,775.310257,0.138875,0.065197,0.111449,22.340623,0.000049,0.010537,...,-19.556418,0.333333,-7.517119,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595


In [36]:
#now for 13 candidates
#read in candidate summary; extract matchids
candidate_summary = pd.read_csv('./processed data/sweeps/r4/field restricted/sweeps_smooth_candidate_summary_field_restricted.csv')
candidate_summary = candidate_summary.iloc[:,1:]
candidate_matchids = candidate_summary['matchid'].values

start_time = time.time()
for i, matchid in enumerate(candidate_matchids):
    current_df = pd.DataFrame(king_kong_data)
    king_kong_candidates = king_kong_candidates.append(populate_king_kong(current_df, matchid), ignore_index=True)
    #print progress
    percent = round((i/len(candidate_matchids))*100, 3)
    print ('progress: ' + str(percent) + '%', end="\r")
end_time = time.time()
king_kong_candidates

,matchid,filter,mad,chi2red,weighted stdev,clipped stdev,iqr,roms,norm excess var,peak to peak var,...,I,Ifi,Iclipped,J,K,L,Jtime,Ltime,Jclipped,Lclipped
0,2868762.0,WFC3_F336W,0.243800,46.826921,0.291822,0.216095,0.399875,7.037212,1.675122e-04,0.015237,...,96.120311,0.666667,0.000000,4.608172,0.751921,4.342711,4.934001,4.649771,41.318034,38.937845
1,2868762.0,WFC3_F438W,0.228700,321.899641,0.352231,0.217952,0.408249,18.044717,2.737215e-04,0.022785,...,39.046318,0.800000,11.284445,4.608172,0.751921,4.342711,4.934001,4.649771,41.318034,38.937845
2,2868762.0,WFC3_F814W,0.145802,17.114714,0.130146,0.094666,0.177502,3.803897,2.787010e-05,0.005772,...,1.441982,0.400000,0.000000,4.608172,0.751921,4.342711,4.934001,4.649771,41.318034,38.937845
3,63916004.0,WFC3_F336W,0.009451,0.055736,0.023585,0.007752,0.014276,0.182833,-1.632574e-05,-0.002659,...,0.167105,0.833333,0.000000,-0.212699,0.687428,-0.183254,0.130665,0.112576,24.694868,21.276198
4,63916004.0,WFC3_F438W,0.021451,0.955049,0.031005,0.021318,0.039226,0.970537,-4.118068e-07,0.000423,...,-0.437264,0.333333,0.000000,-0.212699,0.687428,-0.183254,0.130665,0.112576,24.694868,21.276198
5,63916004.0,WFC3_F814W,0.093301,9.743617,0.104790,0.106019,0.191851,3.698264,2.560430e-05,0.004038,...,-0.086388,0.500000,0.000000,-0.212699,0.687428,-0.183254,0.130665,0.112576,24.694868,21.276198
6,68866566.0,WFC3_F438W,0.263698,137.815454,0.276137,0.201513,0.395799,11.207577,1.351847e-04,0.013960,...,33.075937,0.800000,17.543730,4.744282,0.741123,4.406777,5.113126,4.749381,9.245061,8.587372
7,68866566.0,WFC3_F814W,0.086899,10.970609,0.112066,0.050938,0.089498,2.661473,1.727276e-05,0.004503,...,33.075937,0.800000,17.543730,4.744282,0.741123,4.406777,5.113126,4.749381,9.245061,8.587372
8,3660442.0,WFC3_F438W,0.028299,17.618442,0.072088,0.016425,0.028598,2.979831,1.123033e-05,0.003762,...,12.476706,0.600000,0.000000,1.420852,0.810733,1.443732,0.982085,0.997900,65.543655,66.599093
9,3660442.0,WFC3_F814W,0.116999,81.827284,0.115631,0.114013,0.227999,9.205896,3.107976e-05,0.005637,...,12.476706,0.600000,0.000000,1.420852,0.810733,1.443732,0.982085,0.997900,65.543655,66.599093


In [38]:
#save that mini king kong!
king_kong_candidates.to_csv('./processed data/sweeps/r4/field restricted/king_kong_candidates.csv')

In [45]:
#populate kind kong mega!!
unique_matchids = df['matchid'].unique()
start_time = time.time()
for i, matchid in enumerate(unique_matchids):
    current_df = pd.DataFrame(king_kong_data)
    king_kong_mega = king_kong_mega.append(populate_king_kong(current_df, matchid), ignore_index=True)
    #print progress
    percent = round((i/len(unique_matchids))*100, 3)
    print ('progress: ' + str(percent) + '%', end="\r")
end_time = time.time()
king_kong_mega

,matchid,filter,mad,chi2red,weighted stdev,clipped stdev,iqr,roms,norm excess var,peak to peak var,...,I,Ifi,Iclipped,J,K,L,Jtime,Ltime,Jclipped,Lclipped
0,352.0,ACS_F606W,0.118000,2.291321,0.174979,0.128560,0.330575,1.292976,0.000047,0.008823,...,-2.391036,0.409091,0.0,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
1,352.0,ACS_F814W,0.327699,40.643958,0.303004,0.258211,0.582800,6.762626,0.000203,0.020910,...,-2.391036,0.409091,0.0,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
2,857.0,WFC3_F606W,0.036149,45.247635,0.082396,0.031384,0.069126,4.700127,0.000012,0.007959,...,18.065059,0.833333,0.0,3.120523,0.712588,2.786929,3.224098,2.879431,49.911355,44.575664
3,857.0,WFC3_F814W,0.033401,18.824359,0.048016,0.026966,0.054724,3.505349,0.000005,0.004180,...,18.065059,0.833333,0.0,3.120523,0.712588,2.786929,3.224098,2.879431,49.911355,44.575664
4,875.0,ACS_F435W,0.038400,7.116879,0.051365,0.032218,0.071100,2.339009,0.000004,0.003341,...,18.374893,0.722222,0.0,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179866,108168989.0,ACS_F606W,0.084900,1024.685702,0.097018,0.072621,0.140200,29.304967,0.000026,0.007375,...,132.078348,0.500000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179867,108168989.0,ACS_F658N,0.025400,22.502191,0.058026,0.028015,0.055800,3.995249,0.000007,0.002952,...,142.935457,0.700000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179868,108168989.0,ACS_F814W,0.074049,1388.090054,0.114345,0.098322,0.180150,36.643929,0.000040,0.008087,...,-119.786533,0.500000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179869,108169792.0,WFC3_F606W,0.058799,106.281545,0.064248,0.043672,0.083899,9.316022,0.000008,0.004061,...,0.000000,0.500000,0.0,84.554122,0.795048,84.253522,88.072822,87.759713,84.554122,84.253522


In [46]:
#save that puppy!!
king_kong_mega.to_csv('./processed data/variability indices/king_kong_mega.csv')

## Load in King Kong to plot images from clustering results

In [3]:
king_kong_mega = pd.read_csv('./processed data/variability indices/king_kong_mega.csv')
king_kong_mega = king_kong_mega.iloc[:, 1:]
king_kong_mega

,matchid,filter,mad,chi2red,weighted stdev,clipped stdev,iqr,roms,norm excess var,peak to peak var,...,I,Ifi,Iclipped,J,K,L,Jtime,Ltime,Jclipped,Lclipped
0,352.0,ACS_F606W,0.118000,2.291321,0.174979,0.128560,0.330575,1.292976,0.000047,0.008823,...,-2.391036,0.409091,0.0,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
1,352.0,ACS_F814W,0.327699,40.643958,0.303004,0.258211,0.582800,6.762626,0.000203,0.020910,...,-2.391036,0.409091,0.0,14.086345,0.817772,14.437453,14.575933,14.939244,21.334591,21.866364
2,857.0,WFC3_F606W,0.036149,45.247635,0.082396,0.031384,0.069126,4.700127,0.000012,0.007959,...,18.065059,0.833333,0.0,3.120523,0.712588,2.786929,3.224098,2.879431,49.911355,44.575664
3,857.0,WFC3_F814W,0.033401,18.824359,0.048016,0.026966,0.054724,3.505349,0.000005,0.004180,...,18.065059,0.833333,0.0,3.120523,0.712588,2.786929,3.224098,2.879431,49.911355,44.575664
4,875.0,ACS_F435W,0.038400,7.116879,0.051365,0.032218,0.071100,2.339009,0.000004,0.003341,...,18.374893,0.722222,0.0,44.435443,0.681763,37.968456,68.945457,58.911364,57.484732,49.118595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179866,108168989.0,ACS_F606W,0.084900,1024.685702,0.097018,0.072621,0.140200,29.304967,0.000026,0.007375,...,132.078348,0.500000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179867,108168989.0,ACS_F658N,0.025400,22.502191,0.058026,0.028015,0.055800,3.995249,0.000007,0.002952,...,142.935457,0.700000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179868,108168989.0,ACS_F814W,0.074049,1388.090054,0.114345,0.098322,0.180150,36.643929,0.000040,0.008087,...,-119.786533,0.500000,0.0,74.549859,0.760744,71.079693,139.120399,132.644587,103.768582,98.938335
179869,108169792.0,WFC3_F606W,0.058799,106.281545,0.064248,0.043672,0.083899,9.316022,0.000008,0.004061,...,0.000000,0.500000,0.0,84.554122,0.795048,84.253522,88.072822,87.759713,84.554122,84.253522


In [4]:
columns = list(king_kong_mega.columns)
columns

['matchid',
 'filter',
 'mad',
 'chi2red',
 'weighted stdev',
 'clipped stdev',
 'iqr',
 'roms',
 'norm excess var',
 'peak to peak var',
 'lag1 auto',
 'cssd',
 'Ex',
 'eta ratio',
 'SB',
 'pair1',
 'pair2',
 'I',
 'Ifi',
 'Iclipped',
 'J',
 'K',
 'L',
 'Jtime',
 'Ltime',
 'Jclipped',
 'Lclipped']

In [ ]:
selected_columns = ['mad',
 'chi2red',
 'weighted stdev',
 'clipped stdev',
 'iqr',
 'roms',
 'norm excess var',
 'peak to peak var',
 'Ex',
 'eta ratio',
 'SB',
 'I',
 'Ifi',
 'J',
 'K',
 'Jtime'                  
]